In [1]:
import data
import model
import numpy as np
from random import choice, seed
from model import build_model, dense, dropout, prelu

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.
ERROR:theano.sandbox.cuda:nvcc compiler not found on $PATH. Check your nvcc installation and try again.
Using Theano backend.


In [2]:
X, y, _, _, _ = data.load("~/Documents/flavours-of-physics-start/input/training.csv",
                          shuffle=True)
Xa, ya, wa, _, _ = data.load("~/Documents/flavours-of-physics-start/input/check_agreement.csv",
                             shuffle=False, weight=True)
Xc, _, _, mc, _ = data.load("~/Documents/flavours-of-physics-start/input/check_correlation.csv",
                            shuffle=False, mass=True, test=True)

X, scaler = data.preprocess_data(X)
Xa, _ = data.preprocess_data(Xa, scaler)
Xc, _ = data.preprocess_data(Xc, scaler)

X_test, _, _, _, ids = data.load("~/Documents/flavours-of-physics-start/input/test.csv", test=True,
                                 ids=True)
X_test, _ = data.preprocess_data(X_test, scaler)
print("Done")

Done


In [3]:
def create_model(input_size, l=0.5, domain_loss_weight=0.2):

    model = build_model(input_size,
                        [dense(3 * input_size), prelu(),
                         dense(2 * input_size), prelu(),
                         dense(1.5 * input_size), prelu(),
                         dense(input_size), 
                         prelu(), dropout(0.05), dense(150),
                         prelu(), dropout(0.03), dense(75),
                         prelu(), dropout(0.02), dense(2)],
                        [prelu(), dropout(0.05), dense(100),
                         prelu(), dropout(0.03), dense(75),
                         prelu(), dense(2)], 3, l, domain_loss_weight=domain_loss_weight)
    return model

In [4]:
np.random.seed(1337)  # repeatability
seed(1337)

n_models = 1
n_epochs = 50
probs = None
for i in range(n_models):
    l = 1.6
    domain_weight = 1
    print("Model {}; l = {}; domain_weight = {}".format(i, l, domain_weight))
    m = create_model(X.shape[1], l, domain_weight)
    model.fit_model(m, X, y, Xa, ya, wa, Xc, mc, epoch_count=n_epochs, batch_size=64,
                    validation_split=0.05, verbose=2)
    p = model.predict_model(m, np.array(X_test))
    probs = p if probs is None else p + probs
probs /= n_models

Model 0; l = 1.6; domain_weight = 1
Train on 64175 samples, validate on 3378 samples
Epoch 1/50
KS: 0.05902529951716523 : 0.09 / CvM: 0.0008851526494644607 : 0.002 / AUC: 0.9550188678906582
9s - loss: 1.2613 - val_loss: 1.1217
Epoch 2/50
KS: 0.06805268488096328 : 0.09 / CvM: 0.0009369377641759332 : 0.002 / AUC: 0.9807082808172854
9s - loss: 6.4895 - val_loss: 6.8044
Epoch 3/50
KS: 0.08777376134181367 : 0.09 / CvM: 0.0009399081885834015 : 0.002 / AUC: 0.9864393526154367
9s - loss: 6.5109 - val_loss: 6.5836
Epoch 4/50
KS: 0.1654621565583304 : 0.09 / CvM: 0.0010448282658623808 : 0.002 / AUC: 0.9912938355733196
11s - loss: 6.5029 - val_loss: 6.4849
Epoch 5/50
KS: 0.05940361085104831 : 0.09 / CvM: 0.0010740720346205746 : 0.002 / AUC: 0.9799803438157619
11s - loss: 5.9401 - val_loss: 6.3535
Epoch 6/50
KS: 0.15480424219669697 : 0.09 / CvM: 0.0010131993626380628 : 0.002 / AUC: 0.9912219303492774
11s - loss: 6.4566 - val_loss: 6.4978
Epoch 7/50
KS: 0.19542017307538845 : 0.09 / CvM: 0.0010954637

/usr/local/lib/python3.5/site-packages/keras/initializations.py:30: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return K.variable(np.random.uniform(low=-scale, high=scale, size=shape),
/usr/local/lib/python3.5/site-packages/keras/backend/theano_backend.py:58: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return variable(np.zeros(shape), dtype, name)


KeyboardInterrupt: 

In [ ]:
data.save_submission(ids, probs, "grl_prediction.csv")

In [ ]:
np.random.seed(1337)  # repeatability
seed(1337)

n_models = 100
n_epochs = 100
for i in range(n_models):
    l = np.random.choice(np.linspace(0, 30, 20))
    domain_weight = np.random.choice(np.linspace(0.5, 3, 5))
    print("Model {}; l = {}; domain_weight = {}".format(i, l, domain_weight))
    m = create_model(X.shape[1], l, domain_weight)
    model.fit_model(m, X, y, Xa, ya, wa, Xc, mc, epoch_count=n_epochs, batch_size=64,
                    validation_split=0.05, verbose=0)
    p = model.predict_model(m, np.array(X_test))
    data.save_submission(ids, p, "grl_prediction_{}_{}.csv".format(l, domain_weight))